<h1 align='center'><b><font color='indianred'></p>Spam Detection HW - File 2</b></h1>

<font color = 'indianred' size = 4 > </font>


### <font color = 'blue'>IMPLEMENTING FINAL MODEL TO LARGER DATASET


In [35]:
import sys
import pandas as pd
from pathlib import Path
import joblib

from sklearn.base import TransformerMixin, BaseEstimator

# Import scikit-learn classes for building models
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split


# Import the scipy library for working with sparse matrices
from scipy.sparse import csr_matrix

# for treating class imbalance:
from sklearn.utils import class_weight


In [4]:
import sys
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

    !pip install -U nltk -qq
    !pip install -U spacy -qq
    !python -m spacy download en_core_web_sm -qq

    basepath = '/content/drive/MyDrive/data'
    sys.path.append('/content/drive/MyDrive/data/custom-functions')
else:
    basepath = '/Users/rechitasingh/Downloads/BUAN 6342 - NLP/0_Data'
    sys.path.append(
        '/Users/rechitasingh/Downloads/BUAN 6342 - NLP/0_Python_Scripts')


In [ ]:
# Importing custom functions (Available from class notes)
import custom_preprocessor_mod as cp
from plot_learning_curve import plot_learning_curve
from featurizer import ManualFeatures

In [7]:
# Setting base folder path; keeping all the files in same folder
base_folder = Path(basepath)
data_folder = base_folder
model_folder = base_folder
custom_functions = base_folder

## <font color='blue'>Loading saved Data & Model from Previous Workbook (File 1)</font>

In [53]:
# Setting exact path for all files to be loaded
# NOTE: Train test split is 70%-30% 

path_X_train_org_cleaned = model_folder / \
    'x_train_org_cleaned_sparse_embed.pkl'
path_X_test_org_cleaned = model_folder / \
    'x_test_org_cleaned_sparse_embed.pkl'

file_y_train_org = model_folder / \
    'y_train_org.pkl'
file_y_test_org = model_folder / \
    'y_test_org.pkl'

**Imported the final train and test split from File 1**

   **Note**: X_train_org and X_test_org are preprocessed datasets

In [56]:
X_train_org_cleaned = joblib.load(path_X_train_org_cleaned)
X_test_org_cleaned = joblib.load(path_X_test_org_cleaned)

y_train_org = joblib.load(file_y_train_org)
y_test_org =  joblib.load(file_y_test_org)

In [47]:
# Load the saved model
file_best_estimator_pipeline3 = model_folder / 'pipeline3_best_estimator.pkl'
loaded_model = joblib.load(file_best_estimator_pipeline3)

**Using custom featurizer for FeatureEngineering**

In [27]:
# Using custom featurizer mod
featurizer = ManualFeatures(spacy_model='en_core_web_sm')

X_train_features, feature_names = featurizer.fit_transform(X_train_org_cleaned)

/Users/rechitasingh/Downloads/BUAN 6342 - NLP/0_Python_Scripts/custom_preprocessor_mod.py:90: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


## <font color = 'blue'>Pipeline 3 - Sparse Embeddings (TF-IDF) + Feature Engineering (6 Points)

In [28]:
# Combining text and non text data from Part1 and PArt2

X_train_final = pd.concat((pd.DataFrame(X_train_org_cleaned, columns=['cleaned_text']),
                           pd.DataFrame(X_train_features, columns=feature_names)), axis=1)


In [29]:
X_train_final.head()

,cleaned_text,count_words,count_characters,count_characters_no_space,avg_word_length,count_digits,count_numbers,noun_count,aux_count,verb_count,adj_count,ner
0,goal arsenal 4 henry 7 v liverpool 2 henry sco...,16.0,97.0,82.0,4.823529,7.0,6.0,6.0,15.0,0.0,0.0,1.0
1,not send plus mode,4.0,18.0,15.0,3.000000,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,aah bless arm,3.0,13.0,11.0,2.750000,0.0,0.0,1.0,3.0,0.0,0.0,0.0
3,oh k. come tomorrow,4.0,19.0,16.0,3.200000,0.0,0.0,0.0,2.0,0.0,1.0,0.0
4,yup ok,2.0,6.0,5.0,1.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
class SparseTransformer(TransformerMixin, BaseEstimator):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return csr_matrix(X)


In [36]:
# Creating pipeline for sparse features and vectorizer

sparse_features = Pipeline([('sparse', SparseTransformer()), ])
vectorizer = Pipeline([('tfidf', TfidfVectorizer(max_features=5)), ])

In [37]:

sparse_features.fit_transform(X_train_final.iloc[:, 1:])

<3900x11 sparse matrix of type '<class 'numpy.float64'>'
	with 27774 stored elements in Compressed Sparse Row format>

In [38]:
vectorizer.fit_transform(X_train_final.iloc[:, 0])


<3900x5 sparse matrix of type '<class 'numpy.float64'>'
	with 1041 stored elements in Compressed Sparse Row format>

In [39]:
combined_features = ColumnTransformer(
    transformers=[
        ('tfidf', vectorizer, 'cleaned_text'),
    ], remainder=sparse_features
)


In [40]:
test = combined_features.fit_transform(X_train_final)
print('test.dtype: ', test.dtype)
test

test.dtype:  float64


array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 3.],
       [0., 1., 0., ..., 0., 4., 2.],
       [0., 0., 0., ..., 0., 1., 0.]])

## <font color ='blue'>**Training larger data on best model**</font>

This pipeline will include the pipelines build before for vectorizer and sparcefeatures

In [57]:
# Train the model on the original training data
loaded_model.fit(X_train_final, y_train_org)

Pipeline(steps=[('combined_features',
                 ColumnTransformer(remainder=Pipeline(steps=[('sparse',
                                                              SparseTransformer())]),
                                   transformers=[('tfidf',
                                                  Pipeline(steps=[('tfidf',
                                                                   TfidfVectorizer(max_features=1000))]),
                                                  'cleaned_text')])),
                ('classifier',
                 LogisticRegression(C=10,
                                    class_weight={'ham': 1,
                                                  'spam': 3.5454545454545454},
                                    max_iter=10000))])

In [67]:
# Now the 'loaded_model' contains the trained model with the best parameters
# Using it to make predictions on new data
# Final Pipeline
def final_pipeline(text, classifier=None):
    cleaned_text = text
    X_features, feature_names = featurizer.fit_transform(text)
    X_final = pd.concat((pd.DataFrame(cleaned_text, columns=['cleaned_text']),
                         pd.DataFrame(X_features, columns=feature_names)), axis=1)
    best_estimator_pipeline3 = classifier
    predictions = best_estimator_pipeline3.predict(X_final)
    return predictions

In [68]:
# predicted values for Test data set
y_test_pred3 = final_pipeline(X_test_org_cleaned, loaded_model)


/Users/rechitasingh/Downloads/BUAN 6342 - NLP/0_Python_Scripts/custom_preprocessor_mod.py:90: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


### <font color ='blue'>**Classification report for test dataset**

In [69]:
print('\nTest set classification report:\n\n',
      classification_report(y_test_org, y_test_pred3))



Test set classification report:

               precision    recall  f1-score   support

         ham       0.99      0.99      0.99      1448
        spam       0.96      0.94      0.95       224

    accuracy                           0.99      1672
   macro avg       0.98      0.97      0.97      1672
weighted avg       0.99      0.99      0.99      1672



#### <font color ='blue'> Classification report inference </font>

**Precision:**

1. For the 'ham' class, the precision is 0.99, indicating that 99% of the instances predicted as 'ham' were correctly classified as 'ham.'
2. For the 'spam' class, the precision is 0.96, meaning that 96% of the instances predicted as 'spam' were correctly classified as 'spam.'

**Recall:**

1. For the 'ham' class, the recall is 0.99, indicating that 99% of the actual 'ham' instances were correctly predicted as 'ham.'
2. For the 'spam' class, the recall is 0.94, meaning that 94% of the actual 'spam' instances were correctly predicted as 'spam.'


**F1-Score:**

1. For the 'ham' class, the F1-score is 0.99, which is a very high value, indicating excellent performance in terms of precision and recall.
2. For the 'spam' class, the F1-score is 0.95, which is also a relatively high score but slightly lower than for the 'ham' class.

### <font color ='blue'> Final Inference </font>

The model seem to perform well in the test data. 

**Final model is based out of Pipeline3 - Sparse Embeddings (TF-IDF) + Feature Engineering**

## <font color='blue'> ******************************** THE END- FILE 2 ********************************</font>